In [1]:
import numpy as np
import matplotlib.pyplot as plt
import legwork as lw
import astropy.units as u
import tqdm
from astropy.cosmology import Planck18, z_at_value
from scipy.integrate import trapz, cumtrapz
from schwimmbad import MultiPool
from utils import get_LISA_norm, get_LISA_norm_circular, dg_de
from scipy.interpolate import interp1d

In [2]:
def get_LIGO_rate_uniform_e(m1, n_e_bins):
    if m1 < 20:
        rate = 23.6 / n_e_bins * u.Gpc**(-3) * u.yr**(-1)
    elif m1 < 50:
        rate = 4.5 / n_e_bins * u.Gpc**(-3) * u.yr**(-1)
    elif m1 <= 100:
        rate = 0.2 / n_e_bins * u.Gpc**(-3) * u.yr**(-1)
        
    return rate
        
    
def get_LIGO_rate_iso_dyn(m1, e, frac_iso, ebins):
    e_circ = ebins[e < 1e-6]
    e_ecc = ebins[e >= 1e-6]
    if m1 < 20:
        if e < 1e-6:
            rate = 20 / len(e_circ) * frac_iso * u.Gpc**(-3) * u.yr**(-1)
        else:
            rate = 20 / len(e_ecc) * (1-frac_iso) * u.Gpc**(-3) * u.yr**(-1)
    elif m1 < 50:
        if e < 1e-6:
            rate = 4.5 / len(e_circ) * frac_iso * u.Gpc**(-3) * u.yr**(-1)
        else:
            rate = 4.5 / len(e_circ) * (1-frac_iso) * u.Gpc**(-3) * u.yr**(-1)
    elif m1 <= 100:
        if e < 1e-6:
            rate = 0.2 / len(e_circ) * frac_iso * u.Gpc**(-3) * u.yr**(-1)
        else:
            rate = 0.2 / len(e_ecc) * (1-frac_iso) * u.Gpc**(-3) * u.yr**(-1)
        
    return rate

def ligo_rate(m1):
    dat = np.array([[3.705799151343708, 0.001087789470121345],
                   [4.384724186704389, 0.00984816875074369],
                   [5.063649222065067, 0.06979974252228799],
                   [5.827439886845831, 0.41173514594201527],
                   [6.506364922206512, 1.3579705933006465],
                   [6.845827439886847, 2.148948034692836],
                   [7.77934936350778, 2.7449738151212433],
                   [8.543140028288544, 2.6218307403757986],
                   [9.561527581329564, 2.0525434471508692],
                   [11.173974540311175, 1.2388629239937763],
                   [12.701555869872706, 0.7828664968878465],
                   [14.398868458274404, 0.4947116747780942],
                   [16.859971711456865, 0.2895969742197884],
                   [19.66053748231967, 0.17748817964452962],
                   [22.206506364922213, 0.12773570001722281],
                   [24.837340876944843, 0.10389898279212807],
                   [27.722772277227726, 0.1087789470121345],
                   [30.183875530410184, 0.13070104796093673],
                   [32.729844413012735, 0.16441704701060267],
                   [34.85148514851486, 0.16695189854274867],
                   [37.397454031117405, 0.12107555776371784],
                   [39.26449787835927, 0.08010405199404155],
                   [41.30127298444131, 0.049851062445855264],
                   [43.592644978783596, 0.029631988560550687],
                   [45.629420084865636, 0.018440841322693136],
                   [48.0905233380481, 0.011832859313068754],
                   [50.891089108910904, 0.007949361111716631],
                   [53.77652050919379, 0.005764973856945108],
                   [57.25601131541727, 0.0043438393396653925],
                   [61.923620933521946, 0.0032730313574784275],
                   [66.67609618104669, 0.0024851284269805634],
                   [70.66478076379069, 0.002068305171949823],
                   [74.82319660537483, 0.0016952583040389245],
                   [78.72701555869875, 0.0013476220436441713],
                   [81.27298444130128, 0.0010389898279212807]])
    
    mass = dat[:,0]
    rate = dat[:,1]
    interp_rate = interp1d(mass, rate)
    
    return interp_rate(m1)

In [3]:
import warnings
warnings.filterwarnings("ignore")
n_e_bins = 10
mass1_range = np.logspace(np.log10(5), np.log10(80), 50)
mass2_range = np.logspace(np.log10(5), np.log10(80), 50)
ecc_range = np.logspace(-8, -4, n_e_bins)

e_grid = np.logspace(-7, -4, n_e_bins)
mass1_grid = np.linspace(5, 80, n_e_bins)
mass2_grid = np.linspace(5, 80, n_e_bins)
M, E = np.meshgrid(mass1_grid, n_e_bins)

LISA_norms = []
times = []
ecc_evols = []
f_orb_evols = []
horizon_volumes = []
mass = []
e_LIGO = []
for m1 in tqdm.tqdm(mass1_range):
    for e in ecc_range:
        dat = [m1, m1, e]
        f_orb_evol, ecc_evol, timesteps, LISA_norm = get_LISA_norm(dat)
        print(f_orb_evol)
        ind, = np.where(f_orb_evol < 0.1 * u.Hz)
                
        source = lw.source.Source(m_1=m1*u.Msun * np.ones_like(ecc_evol[ind]),
                                  m_2=m1*u.Msun * np.ones_like(ecc_evol[ind]),
                                  ecc=ecc_evol[ind],
                                  f_orb=f_orb_evol[ind],
                                  dist=8 * np.ones_like(ecc_evol[ind]) * u.Mpc,
                                  interpolate_g=False,
                                  n_proc=2,
                                 )
        
        snr = source.get_snr(approximate_R=True)
        D_h = snr/7 * 8 * u.Mpc
        print(D_h)
        redshift = np.ones(len(D_h)) * 1e-8
        redshift[D_h > 1 * u.kpc] = z_at_value(Planck18.luminosity_distance, D_h[D_h > 1 * u.kpc])
        V_c = Planck18.comoving_volume(z=redshift)
        horizon_volumes.append(V_c)
        mass.append(m1)
        e_LIGO.append(e)
        LISA_norms.append(LISA_norm[ind])
        times.append(timesteps[ind])
        ecc_evols.append(ecc_evol[ind])
        f_orb_evols.append(f_orb_evol[ind])
        

  0%|                                                    | 0/50 [00:00<?, ?it/s]


NameError: name 'freq_evol' is not defined

In [ ]:
LIGO_rate_uniform = []
#LIGO_rate_iso_dyn_50 = []
#LIGO_rate_iso_dyn_80 = []

for m1 in tqdm.tqdm(mass1_range):
    for m2 in mass2_range:
        if m2 < m1:
            for e in ecc_range:
                LIGO_rate_uniform.append(ligo_rate(m1)/len(ecc_range))
                #LIGO_rate_iso_dyn_50.append(get_LIGO_rate_iso_dyn(m1, e, frac_iso=0.5, ebins=ecc_range))
                #LIGO_rate_iso_dyn_80.append(get_LIGO_rate_iso_dyn(m1, e, frac_iso=0.8, ebins=ecc_range))
                

In [ ]:
N_lisa_tot_uniform = []
#N_lisa_tot_iso_dyn_50 = []
#N_lisa_tot_iso_dyn_80 = []
for ii in range(len(LISA_norms)):

    N_lisa_tot_uniform.append(trapz((LISA_norms[ii]*LIGO_rate_uniform[ii]*V_c[ii]).to(u.Hz**(-1))).value, f_orb_evols[ii])
    #N_lisa_tot_iso_dyn_50.append(trapz((LISA_norms[ii]*LIGO_rate_iso_dyn_50[ii]*V_c[ii]).to(u.Hz**(-1))).value, f_orb_evols[ii])
    #N_lisa_tot_iso_dyn_80.append(trapz((LISA_norms[ii]*LIGO_rate_iso_dyn_80[ii]*V_c[ii]).to(u.Hz**(-1))).value, f_orb_evols[ii])

In [ ]:
M_c = lw.utils.chirp_mass(M1*u.Msun, M2*u.Msun)

In [ ]:
fig = plt.figure(figsize=(6, 4))
rate_tot = 0
for m, e, l, LR, f, V in zip(M_c, ecc_evols, LISA_norms,LIGO_rate_uniform, f_orb_evols, V_c):
    rate_tot += m * trapz(f, (l * LR * V).to(u.Hz**(-1)))
    plt.scatter(f, cumtrapz(f, (l * LR ).to(u.Hz**(-1)*u.Mpc**(-3)) * V, initial=0), c=np.log10(e), label=np.round(m, 2), s=5)
plt.xscale('log')
plt.yscale('log')
plt.colorbar()
print(rate_tot)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
ax1.scatter(M_c, E, c=N_lisa_tot_uniform, s=M2, norm=colors.LogNorm())
#print(np.sum(N_lisa_tot_uniform))
ax1.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_uniform), 4)}')
ax2.scatter(M_c, E, c=N_lisa_tot_iso_dyn_50, s=M2, norm=colors.LogNorm())
ax2.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_iso_dyn_50), 4)}')
c = ax3.scatter(M_c, E, c=N_lisa_tot_iso_dyn_80, s=M2, norm=colors.LogNorm())
ax3.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_iso_dyn_80), 4)}')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax3.set_yscale('log')
plt.colorbar(c)    

In [ ]:
## let's try it out as just circular for now...

In [ ]:
n_grid = 50
nproc=1
#e_grid = np.logspace(-9, -4, n_grid)
mass_grid = np.linspace(5, 80, n_grid)
m_c = lw.utils.chirp_mass(mass_grid * u.Msun, mass_grid * u.Msun)
#M1, M2= np.meshgrid(mass_grid, mass_grid)
E = np.zeros_like(mass_grid)

In [ ]:
with MultiPool(processes=nproc) as pool:
    dat_out = list(pool.map(get_LISA_norm_circular, zip(list(mass_grid), list(mass_grid), list(E))))


In [ ]:
V_c = []
LIGO_rate_uniform = []
LIGO_rate_iso_dyn_50 = []
LIGO_rate_iso_dyn_80 = []
LIGO_rate = []
times = []
ecc_evols = []
f_orb_evols = []
LISA_norms = []
m1_evols = []
m2_evols = []

for d, m1, m2, e in tqdm.tqdm(zip(dat_out, mass_grid, mass_grid, E), total=len(mass_grid)):
    f_orb_evol, ecc_evol, timesteps, LISA_norm = d
    
    LISA_norms.append(LISA_norm.to(u.yr/u.Hz))
    times.append(timesteps)
    ecc_evols.append(ecc_evol)
    f_orb_evols.append(f_orb_evol)
    m1_evols.append(m1 * np.ones(len(f_orb_evol)))
    m2_evols.append(m2 * np.ones(len(f_orb_evol)))
    LIGO_rate_uniform.append(get_LIGO_rate_uniform_e(m1, n_grid))
    LIGO_rate_iso_dyn_50.append(get_LIGO_rate_iso_dyn(m1, e, frac_iso=0.5))
    LIGO_rate_iso_dyn_80.append(get_LIGO_rate_iso_dyn(m1, e, frac_iso=0.8))
    LIGO_rate.append(ligo_rate(m1))

times = np.array(times)
ecc_evols = np.array(ecc_evols)
f_orb_evols = np.array(f_orb_evols)
m1_evols = np.array(m1_evols)
m2_evols = np.array(m2_evols)


In [ ]:
np.shape(m1_evols), np.shape(f_orb_evols)

In [ ]:
source = lw.source.Source(m_1=m1_evols.flatten() * u.Msun,
                          m_2=m2_evols.flatten() * u.Msun,
                          ecc=ecc_evols.flatten(),
                          f_orb=f_orb_evols.flatten() * u.Hz,
                          dist=8 * np.ones(len(f_orb_evols.flatten())) * u.Mpc,
                          interpolate_g=False,
                          n_proc=1)
snr = source.get_snr(approximate_R=True, verbose=True)
D_h = snr/7 * 8 * u.Mpc
redshift = np.ones(len(D_h)) * 1e-8
redshift[D_h > 0.0001 * u.Mpc] = z_at_value(Planck18.luminosity_distance, D_h[D_h > 0.0001 * u.Mpc])
V_c = Planck18.comoving_volume(z=redshift)


In [ ]:
V_c_reshape = V_c.reshape(f_orb_evols.shape)
SNR_reshape = snr.reshape(f_orb_evols.shape)

In [ ]:
np.shape(V_c_reshape), np.shape(LIGO_rate), np.shape(mass_grid), np.shape(f_orb_evols)

In [ ]:
V_c_reshape

In [ ]:
for ii in range(len(mass_grid)):
    plt.scatter(f_orb_evols[ii,:], np.ones(100) * m_c[ii], c=np.log10(V_c_reshape[ii,:].value), vmin=-13, vmax=10)
    
plt.xscale('log')
plt.colorbar(label=r'comoving volume [Mpc$^3$]')
plt.xlabel('orbital frequency')
plt.ylabel(r'chirp mass [M$_{\odot}$]; q=1')

In [ ]:
for ii in range(len(mass_grid)):
    rate_per_freq = (V_c_reshape[ii,:] * LISA_norms[ii] * LIGO_rate[ii] * u.Gpc**(-3) * u.yr**(-1)).to(u.Hz**(-1))
    
    plt.scatter(f_orb_evols[ii,:], np.ones(100) * m_c[ii], 
                c=np.log10(rate_per_freq.value))
    
plt.xscale('log')
plt.colorbar(label=r'rate per frequency [Hz$^{-1}$]')
plt.xlabel('orbital frequency')
plt.ylabel(r'chirp mass [M$_{\odot}$]; q=1')

In [ ]:
rate = []
for ii, m1, m2 in zip(range(len(mass_grid)), mass_grid, mass_grid):
    f = f_orb_evols[ii, :]
    v_c = V_c_reshape[ii, :]
    snr = SNR_reshape[ii, :]
    l_norm = LISA_norms[ii]
    l_rate = LIGO_rate[ii] * u.Gpc**(-3) * u.yr**(-1)
    rate.append(trapz(l_norm * v_c * l_rate.to(u.Mpc**(-3) * u.yr**(-1)), f * u.Hz).value)

In [ ]:
print(rate)

In [ ]:
plt.scatter(m_c, rate)

In [ ]:
f_orb_evols.flatten()[np.isnan(snr)]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
ax1.scatter(M_c, E, c=N_lisa_tot_uniform, s=M2, norm=colors.LogNorm())
#print(np.sum(N_lisa_tot_uniform))
ax1.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_uniform), 4)}')
ax2.scatter(M_c, E, c=N_lisa_tot_iso_dyn_50, s=M2, norm=colors.LogNorm())
ax2.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_iso_dyn_50), 4)}')
c = ax3.scatter(M_c, E, c=N_lisa_tot_iso_dyn_80, s=M2, norm=colors.LogNorm())
ax3.set_title(f'N LISA {np.round(np.sum(N_lisa_tot_iso_dyn_80), 4)}')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax3.set_yscale('log')
ax1.set_xscale('log')
ax2.set_xscale('log')
ax3.set_xscale('log')

plt.colorbar(c)    

In [ ]:
plt.scatter(e_LIGO, N_lisa_tot_uniform)
plt.xscale('log')
plt.yscale('log')

In [ ]:
sum(N_lisa_tot_uniform), sum(N_lisa_tot_iso_dyn_50), sum(N_lisa_tot_iso_dyn_80)